### Preparing data for multi-label image classification

Each original chest x-ray image is 1024×1024 in size. We used 224×224 input resolution to obtain initial results, and then we obtained results on the full resolution. The input dataset is split into training/validation and testing without patient overlap between these sets and the 14 diseases are modeled as 14 binary outputs of a multi-label classifier. Each disease output indicates the presence/absence of the disease and “no disease” is represented as zeros for all labels. We first split the data into three sets: training, validation, and testing, based on patient IDs.

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import boto3

import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()
bucket='analytics-serverless-west'
prefix = 'sagemaker/x-ray'

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

        
def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)

def download_from_s3(bucket, key, local_file_name):
    s3 = boto3.resource('s3')
    s3.Bucket(bucket).download_file(key, local_file_name)

trainper = 0.7
valper = 0.1
file_name = 'Data_Entry_2017.csv'

download_from_s3(bucket, prefix+'/'+file_name, file_name)

a = pd.read_csv(file_name)
patient_ids = a['Patient ID']
uniq_pids = np.unique(patient_ids)
np.random.shuffle(uniq_pids)
total_ids = len(uniq_pids)
#total_ids = 30000 #pick a sample size you want to use to train and validate the model

In [ ]:
a.head()

### Make Train, Validation, and Testing datasets from the full image list

In [ ]:
trainset = int(trainper*total_ids)
valset = trainset+int(valper*total_ids)
testset = trainset+valset

train = uniq_pids[:trainset]
val = uniq_pids[trainset+1:valset]
test = uniq_pids[valset+1:]
print('Number of patient ids: training: %d, validation: %d, testing: %d'%(len(train), len(val), len(test)))

traindata = a.loc[a['Patient ID'].isin(train)]
valdata = a.loc[a['Patient ID'].isin(val)]
testdata = a.loc[a['Patient ID'].isin(test)]

traindata.to_csv('traindata.csv', sep=',', header=False, index=False)
valdata.to_csv('valdata.csv', sep=',', header=False, index=False)
testdata.to_csv('testdata.csv', sep=',', header=False, index=False)

Then, we create the ‘[lst](https://mxnet.incubator.apache.org/faq/recordio.html)’ files for multi-label classification where each disease is mapped to a set of binary labels.

In [ ]:
import csv

def gen_set(csvfile, outputfile):
    disease_list = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', \
                   'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', \
                   'Hernia']
    alldiseases = {disease:i for i,disease in enumerate(disease_list)}
    with open(outputfile, 'w') as fp:
        with open(csvfile, 'r') as cfile:
            line = csv.reader(cfile, delimiter=',')
            index = 0
            for element in line:
                # the first column is the image filename, while the second
                # column has the list of diseases separated by |
                diseases = element[1].split('|')
                fp.write('%d\t'%index)
                for d in alldiseases:
                    if d in diseases:
                        fp.write('%d\t'%1)
                    else:
                        fp.write('%d\t'%0)
                fp.write('images/%s\n' % element[0])
                index += 1
                 
gen_set('traindata.csv', 'chestxraytrain.lst')
gen_set('valdata.csv', 'chestxrayval.lst')
gen_set('testdata.csv', 'chestxraytest.lst')      

In [ ]:
t_csv = pd.read_csv('traindata.csv')
v_csv = pd.read_csv('valdata.csv')

t_csv.head()

In [ ]:
v_csv.head()

Then we create recordio files using [im2rec.py](https://github.com/apache/incubator-mxnet/blob/master/tools/im2rec.py) for training and validation. We pass the option pack_label so that the recordio file is created as a multi-label file. For more details, please refer to the Amazon SageMaker multi-label image classification notebooks.

In [ ]:
download('https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py')

### Only run the below cell once. This is an expensive operation

In [ ]:
!aws s3 sync s3://analytics-serverless-west/sagemaker/x-ray/images/ images/

In [ ]:
!head -10 chestxraytest.lst

In [ ]:
!python im2rec.py --pack-label chestxraytrain.lst .
!python im2rec.py --pack-label chestxrayval.lst .

### Upload to S3

In [ ]:
import boto3
import os

bucket = 'analytics-serverless-west'
prefix = 'sagemaker/x-ray'

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', 'chestxraytrain.rec')).upload_file('chestxraytrain.rec')
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print('uploaded training data location: {}'.format(s3_train_data))

boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation', 'chestxrayval.rec')).upload_file('chestxrayval.rec')
s3_validation_data = 's3://{}/{}/validation/{}'.format(bucket, prefix, key)
print('uploaded validation data location: {}'.format(s3_validation_data))